<font color=blue, size="10">**NEWS PUBLISHER**</font>

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import _datetime
import datetime
import time
import calendar
import glob
import matplotlib as mp
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import seaborn as sns
pd.options.display.max_colwidth = 100
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.float_format = '{:,.0f}'.format
from pandas.tseries.offsets import MonthEnd
from google.cloud import bigquery


# For connecting to google sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
# Change this line to point to your own service token file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/dicharryd/service_token.json"
# Configure the connection 
scope = ['https://spreadsheets.google.com/feeds']
# Give the path to the Service Account Credential json file 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/dicharryd/service_token.json',scope)
# Authorise your Notebook
gc = gspread.authorize(credentials)
# spreadsheet ID from URL
spreadsheet_key = '1VpimYGT1nZ20KmsvR6ILs1dIkC4mlDE_0uMsz6ckCDk' #"NEW weekly features"
#make sure sheet is shared with bergstresser@dj-users.iam.gserviceaccount.com

#************************************************************************

#Connecting to BigQuery
client = bigquery.Client()

def query_calls(client, query):
    job_config = bigquery.QueryJobConfig()
    job_config.use_legacy_sql = False
    query_job = client.query(query, job_config=job_config)
    results = query_job.result()
    return results

def job_setup(client, query, dataset, table, write_disposition):
    job_config = bigquery.QueryJobConfig()
    table_ref = client.dataset(dataset).table(table)
    job_config.destination = table_ref
    job_config.use_legacy_sql = False
    job_config.write_disposition = write_disposition
   
    query_job = client.query(
    query,
    location='US',
    job_config=job_config)

    query_job.result()
    print('Query results loaded to table {}'.format(table_ref.path))
    print('Query last completed at {}'.format(datetime.datetime.now()))
    last_modified = client.get_table(dataset + '.' + table).modified
    print('Table last modified at {} GMT timezone'.format(last_modified))

# REFERRAL TABLE

In [2]:
# query = '''

# select channel,
# CASE WHEN news_tab = 1 THEN "News Tab"
# when referrer = "googleapis.com" THEN "Google"
# when referrer LIKE "google.%" THEN "Google"
# ELSE NET.REG_DOMAIN(referrer) END as referrer, 
# -- CASE WHEN channel LIKE "%arket%" THEN "MarketWatch" 
# -- ELSE NET.REG_DOMAIN(referrer) END as new_channel,
# sum(views) as views, 
# count(distinct unique_id) as uniques from (
#         SELECT
#         DISTINCT NET.REG_DOMAIN(visit_referrer) as referrer,
#         DATE(date_time) as read_time,
#         IF(REGEXP_CONTAINS(page_url, 'utm_medium=news_tab') AND NET.REG_DOMAIN(visit_referrer) = 'facebook.com', 1, 0) as news_tab,
#         #IF(REGEXP_CONTAINS(post_evar56, 'personalized'), 1, 0) as locked,
#         IF(post_page_event = '0' AND (CAST(exclude_hit AS INT64) <= 0 AND hit_source NOT IN ('5', '8', '9')),1,0) as views,
#         #IF(post_prop27 IN ('WSJ_sub_yes'), 1, 0) as sub_status,
#         CONCAT(post_visid_high, post_visid_low) as unique_id,
#         visit_num,
#         channel,
#         #IF(prop1 = 'Article', 1, 0) as article,
#         #visit_page_num,
#         #IF(REGEXP_CONTAINS(visit_start_pagename, 'Article'), 1, 0) as article_visit,
#         #IF(page_event_var2 = 'WSJ_Article_social_share', 1, 0) as share,
#         #prop10
        
#         FROM
#         `djomniture.cipomniture_djglobal.*` 
#         WHERE _TABLE_SUFFIX BETWEEN '2021_12' AND '2022_12'
#         #and DATE(TIMESTAMP(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01'))) BETWEEN "2021-12-01" AND "2021-12-31"               
#         #and DATE(date_time) BETWEEN DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -7 DAY), DAY)) AND DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -1 DAY), DAY))    
#         and DATE(date_time) BETWEEN DATE_ADD(DATE_TRUNC(CURRENT_DATE(), WEEK(SUNDAY)), INTERVAL -6 DAY) AND DATE_TRUNC(CURRENT_DATE(), WEEK(SUNDAY))
#         and SAFE_CAST(exclude_hit AS INT64) <= 0
#         AND hit_source NOT IN ('5',
#                                 '8',
#                                 '9')
#         #AND (REGEXP_CONTAINS(page_url, 'wsj[.com|.net]')
#         #and channel IN ('Online Journal')
# ) 
# where (referrer IN ("apple.news", 'facebook.com','news_tab','google.com') OR (referrer LIKE "google.%" OR referrer = "googleapis.com"))
# group by channel, referrer
# order by channel, referrer 

# '''

# referralthisweek = query_calls(client, query).to_dataframe()
# referralthisweek


In [3]:
referral_query1 = '''
select * from (

select channel,
CASE WHEN news_tab = 1 THEN "News Tab"
when referrer = "googleapis.com" THEN "Google"
when referrer LIKE "google.%" THEN "Google"
ELSE NET.REG_DOMAIN(referrer) END as referrer, 
-- CASE WHEN channel LIKE "%arket%" THEN "MarketWatch" 
-- ELSE NET.REG_DOMAIN(referrer) END as new_channel,
sum(views) as views, 
count(distinct unique_id) as uniques from (
        SELECT
        DISTINCT NET.REG_DOMAIN(visit_referrer) as referrer,
        DATE(date_time) as read_time,
        IF(REGEXP_CONTAINS(page_url, 'utm_medium=news_tab') AND NET.REG_DOMAIN(visit_referrer) = 'facebook.com', 1, 0) as news_tab,
        #IF(REGEXP_CONTAINS(post_evar56, 'personalized'), 1, 0) as locked,
        IF(post_page_event = '0' AND (CAST(exclude_hit AS INT64) <= 0 AND hit_source NOT IN ('5', '8', '9')),1,0) as views,
        #IF(post_prop27 IN ('WSJ_sub_yes'), 1, 0) as sub_status,
        CONCAT(post_visid_high, post_visid_low) as unique_id,
        visit_num,
        channel,
        #IF(prop1 = 'Article', 1, 0) as article,
        #visit_page_num,
        #IF(REGEXP_CONTAINS(visit_start_pagename, 'Article'), 1, 0) as article_visit,
        #IF(page_event_var2 = 'WSJ_Article_social_share', 1, 0) as share,
        #prop10
        
        FROM
        `djomniture.cipomniture_djglobal.*` 
        WHERE _TABLE_SUFFIX BETWEEN '2021_12' AND '2022_12'
        #and DATE(TIMESTAMP(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01'))) BETWEEN "2021-12-01" AND "2021-12-31"               
        #and DATE(date_time) BETWEEN DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -7 DAY), DAY)) AND DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -1 DAY), DAY))    
        and DATE(date_time) BETWEEN DATE_ADD(DATE_TRUNC(CURRENT_DATE(), WEEK(SUNDAY)), INTERVAL -6 DAY) AND DATE_TRUNC(CURRENT_DATE(), WEEK(SUNDAY))
        and SAFE_CAST(exclude_hit AS INT64) <= 0
        AND hit_source NOT IN ('5',
                                '8',
                                '9')
        #AND (REGEXP_CONTAINS(page_url, 'wsj[.com|.net]')
        #and channel IN ('Online Journal')
) 
#where (referrer IN ("apple.news", 'facebook.com','news_tab','google.com') OR (referrer LIKE "google.%" OR referrer = "googleapis.com"))
group by channel, referrer
order by views DESC

) where uniques >1000
order by uniques DESC
'''
referralthisweek = query_calls(client, referral_query1).to_dataframe()

referralthisweek['channel'] =    pd.np.where(referralthisweek['channel'].str.contains("arket"),"MarketWatch", 
                            #pd.np.where(referralthisweek['channel'].str.contains("Online Journal"),"WSJ", 
                            #pd.np.where(referralthisweek['channel'].str.contains("WSJ"),"WSJ",
                            pd.np.where(referralthisweek['channel'].str.contains("wsj-video"),"Online Journal",
                            pd.np.where(referralthisweek['channel'].str.contains("barrons-video"),"Barrons", 
                            pd.np.where(referralthisweek['channel'].str.contains("Barrons Online"),"Barrons",
                                                                               referralthisweek['channel']))))

referralthisweek = referralthisweek.loc[referralthisweek['channel'].isin(['Barrons','Online Journal','MarketWatch'])]
referralthisweek = referralthisweek.groupby(['channel','referrer']).sum().reset_index()
referralthisweek.insert(1, 'Week', 'thisweek') 

referralthisweek['referrer'] = pd.np.where(referralthisweek['referrer'].str.contains("goog"),"Google",
                          pd.np.where(referralthisweek['referrer'].str.contains("android.gm"),"Google",
                                     pd.np.where(referralthisweek['referrer']=="facebook.com","Facebook",
                                         pd.np.where(referralthisweek['referrer']=="News Tab","FB News",
                             pd.np.where(referralthisweek['referrer'].str.contains("twitter"),"Twitter",
                                             pd.np.where(referralthisweek['referrer']=="t.co","Twitter",
                                pd.np.where(referralthisweek['referrer'].str.contains("lnkd"),"LinkedIn",
                            pd.np.where(referralthisweek['referrer'].str.contains("linkedin"),"LinkedIn",
                                     pd.np.where(referralthisweek['referrer']=="linkedin.com","LinkedIn",
                               pd.np.where(referralthisweek['referrer'].str.contains("yahoo"),"Yahoo",
                                          pd.np.where(referralthisweek['referrer']=="wsj.com","Direct",
                           pd.np.where(referralthisweek['referrer'].str.contains("robinhood"),"Robinhood",
                             pd.np.where(referralthisweek['referrer'].str.contains("wsj_RHF"),"Robinhood",
                                       pd.np.where(referralthisweek['referrer']=="apple.news","Apple News",
                                    pd.np.where(referralthisweek['referrer']=="instagram.com","Instagram",
                                 pd.np.where(referralthisweek['referrer']=="drudgereport.com","Drudge Report",
                                   pd.np.where(referralthisweek['referrer']=="ampproject.org","Google",
                                         pd.np.where(referralthisweek['referrer']=="bing.com","Bing",
                                                                             referralthisweek['referrer']))))))))))))))))))
referralthisweek = referralthisweek.groupby(['channel','Week','referrer']).sum().sort_values(by='uniques',ascending=False).reset_index()
############################################################################

referral_query2 = '''
select * from (

select channel,
CASE WHEN news_tab = 1 THEN "News Tab"
when referrer = "googleapis.com" THEN "Google"
when referrer LIKE "google.%" THEN "Google"
ELSE NET.REG_DOMAIN(referrer) END as referrer, 
-- CASE WHEN channel LIKE "%arket%" THEN "MarketWatch" 
-- ELSE NET.REG_DOMAIN(referrer) END as new_channel,
sum(views) as views, 
count(distinct unique_id) as uniques from (
        SELECT
        DISTINCT NET.REG_DOMAIN(visit_referrer) as referrer,
        DATE(date_time) as read_time,
        IF(REGEXP_CONTAINS(page_url, 'utm_medium=news_tab') AND NET.REG_DOMAIN(visit_referrer) = 'facebook.com', 1, 0) as news_tab,
        #IF(REGEXP_CONTAINS(post_evar56, 'personalized'), 1, 0) as locked,
        IF(post_page_event = '0' AND (CAST(exclude_hit AS INT64) <= 0 AND hit_source NOT IN ('5', '8', '9')),1,0) as views,
        #IF(post_prop27 IN ('WSJ_sub_yes'), 1, 0) as sub_status,
        CONCAT(post_visid_high, post_visid_low) as unique_id,
        visit_num,
        channel,
        #IF(prop1 = 'Article', 1, 0) as article,
        #visit_page_num,
        #IF(REGEXP_CONTAINS(visit_start_pagename, 'Article'), 1, 0) as article_visit,
        #IF(page_event_var2 = 'WSJ_Article_social_share', 1, 0) as share,
        #prop10
        
        FROM
        `djomniture.cipomniture_djglobal.*` 
        WHERE _TABLE_SUFFIX BETWEEN '2021_12' AND '2022_12'
        #and DATE(TIMESTAMP(CONCAT(SUBSTR(_TABLE_SUFFIX,1,4),'-',SUBSTR(_TABLE_SUFFIX,-2),'-','01'))) BETWEEN "2021-12-01" AND "2021-12-31"               
        #and DATE(date_time) BETWEEN DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -14 DAY), DAY)) AND DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -8 DAY), DAY))    
        and DATE(date_time) between DATE_ADD(DATE_TRUNC(CURRENT_DATE(), WEEK(SUNDAY)), INTERVAL -13 DAY) AND DATE_ADD(DATE_TRUNC(CURRENT_DATE(), WEEK(SUNDAY)), INTERVAL -7 DAY)        
        and SAFE_CAST(exclude_hit AS INT64) <= 0
        AND hit_source NOT IN ('5',
                                '8',
                                '9')
        #AND (REGEXP_CONTAINS(page_url, 'wsj[.com|.net]')
        #and channel IN ('Online Journal')
) 
#where (referrer IN ("apple.news", 'facebook.com','news_tab','google.com') OR (referrer LIKE "google.%" OR referrer = "googleapis.com"))
group by channel, referrer
order by views DESC 

) where uniques >1000
order by uniques DESC
'''
referralpriorweek = query_calls(client, referral_query2).to_dataframe()
referralpriorweek['channel'] =    pd.np.where(referralpriorweek['channel'].str.contains("arket"),"MarketWatch", 
                            pd.np.where(referralpriorweek['channel'].str.contains("wsj-video"),"Online Journal",
                            pd.np.where(referralpriorweek['channel'].str.contains("barrons-video"),"Barrons", 
                            pd.np.where(referralpriorweek['channel'].str.contains("Barrons Online"),"Barrons", 
                            referralpriorweek['channel']))))
referralpriorweek = referralpriorweek.loc[referralpriorweek['channel'].isin(['Barrons','Online Journal','MarketWatch'])]
referralpriorweek = referralpriorweek.groupby(['channel','referrer']).sum().reset_index()
referralpriorweek.insert(1, 'Week', 'priorweek') 

referralpriorweek['referrer'] = pd.np.where(referralpriorweek['referrer'].str.contains("goog"),"Google",
                          pd.np.where(referralpriorweek['referrer'].str.contains("android.gm"),"Google",
                                     pd.np.where(referralpriorweek['referrer']=="facebook.com","Facebook",
                                         pd.np.where(referralpriorweek['referrer']=="News Tab","FB News",
                             pd.np.where(referralpriorweek['referrer'].str.contains("twitter"),"Twitter",
                                             pd.np.where(referralpriorweek['referrer']=="t.co","Twitter",
                                pd.np.where(referralpriorweek['referrer'].str.contains("lnkd"),"LinkedIn",
                            pd.np.where(referralpriorweek['referrer'].str.contains("linkedin"),"LinkedIn",
                                     pd.np.where(referralpriorweek['referrer']=="linkedin.com","LinkedIn",
                               pd.np.where(referralpriorweek['referrer'].str.contains("yahoo"),"Yahoo",
                                          pd.np.where(referralpriorweek['referrer']=="wsj.com","Direct",
                           pd.np.where(referralpriorweek['referrer'].str.contains("robinhood"),"Robinhood",
                             pd.np.where(referralpriorweek['referrer'].str.contains("wsj_RHF"),"Robinhood",
                                       pd.np.where(referralpriorweek['referrer']=="apple.news","Apple News",
                                    pd.np.where(referralpriorweek['referrer']=="instagram.com","Instagram",
                                 pd.np.where(referralpriorweek['referrer']=="drudgereport.com","Drudge Report",
                                   pd.np.where(referralpriorweek['referrer']=="ampproject.org","Google",
                                         pd.np.where(referralpriorweek['referrer']=="bing.com","Bing",
                                                                             referralpriorweek['referrer']))))))))))))))))))
referralpriorweek = referralpriorweek.groupby(['channel','Week','referrer']).sum().sort_values(by='uniques',ascending=False).reset_index()
############################################################################
combined = pd.concat([referralthisweek,referralpriorweek], sort=False).reset_index()
############################################################################
# Set the sheet name you want to upload data to and the start cell where the upload data begins 
# wks_name = 'referral'
# cell_of_start_df = 'a1'
# # upload the dataframe of the clients we want to delete
# d2g.upload(combined,
#            spreadsheet_key,
#            wks_name,
#            credentials=credentials,
#            col_names=True,
#            row_names=False,
#            start_cell = cell_of_start_df,
#            clean=True)
# print ('The sheet is updated successfully')
combined


/var/folders/y6/ncpp7cyx1jz11rm61jlv5js80000gq/T/ipykernel_48862/1929484052.py:51: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  referralthisweek['channel'] =    pd.np.where(referralthisweek['channel'].str.contains("arket"),"MarketWatch",
/var/folders/y6/ncpp7cyx1jz11rm61jlv5js80000gq/T/ipykernel_48862/1929484052.py:54: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  pd.np.where(referralthisweek['channel'].str.contains("wsj-video"),"Online Journal",
/var/folders/y6/ncpp7cyx1jz11rm61jlv5js80000gq/T/ipykernel_48862/1929484052.py:55: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  pd.np.where(referralthisweek['channel'].str.contains("barrons-video"),"Barrons",
/var/folders/y6/ncpp7cyx1jz11rm61jlv5js80000gq/T/ipykernel_48862/192948

,index,channel,Week,referrer,views,uniques
0,0,MarketWatch,thisweek,Google,6687936,5509128
1,1,Online Journal,thisweek,Google,5725380,5170041
2,2,MarketWatch,thisweek,Yahoo,2601848,2099939
3,3,Online Journal,thisweek,Facebook,1603922,1433126
4,4,Online Journal,thisweek,Direct,2249301,1262406
...,...,...,...,...,...,...
269,129,MarketWatch,priorweek,schoology.com,1684,1049
270,130,Online Journal,priorweek,appspace.com,1047,1047
271,131,Online Journal,priorweek,vanguard.com,3060,1037
272,132,Online Journal,priorweek,stevequayle.com,1033,1020


# WEBSTORIES

In [82]:
#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key('1LeCZ7-3E-tZyd9OAYAqHIOF-y4EIpnE0eKllL1wE6ks')
sheet = workbook.worksheet('Landing Articles (All)')
values = sheet.get_all_values()
webstories = pd.DataFrame(values[1:], columns = values[0])
webstories['Day'] = pd.to_datetime(webstories['Day'])
webstories['Pageviews'] = webstories['Pageviews'].astype(int)
############################################################################
webstories_thisweek = webstories.copy(deep=True)
start_date = ((datetime.datetime.now() - timedelta(days=datetime.datetime.now().weekday() +1)) - timedelta(days=7))
end_date = (datetime.datetime.now() - timedelta(days=datetime.datetime.now().weekday() +1))
mask = (webstories_thisweek['Day'] > start_date) & (webstories_thisweek['Day'] <= end_date) 
webstories_thisweek = webstories_thisweek.loc[mask]
webstories_thisweek = webstories_thisweek.groupby('Product').sum().reset_index()
webstories_thisweek.insert(0, 'Week', 'thisweek') 
############################################################################
webstories_priorweek = webstories.copy(deep=True)
start_date = ((datetime.datetime.now() - timedelta(days=datetime.datetime.now().weekday() +1)) - timedelta(days=14))
end_date =   ((datetime.datetime.now() - timedelta(days=datetime.datetime.now().weekday() +1)) - timedelta(days=7))
mask = (webstories_priorweek['Day'] > start_date) & (webstories_priorweek['Day'] <= end_date) 
webstories_priorweek = webstories_priorweek.loc[mask]
webstories_priorweek = webstories_priorweek.groupby('Product').sum().reset_index()
webstories_priorweek.insert(0, 'Week', 'priorweek') 
############################################################################
webstories_combined = pd.concat([webstories_thisweek,webstories_priorweek], sort=False).reset_index()
webstories_combined['Product'] =    pd.np.where(webstories_combined['Product'].str.contains("WSJ"),"Online Journal", webstories_combined['Product'])
############################################################################
# Set the sheet name you want to upload data to and the start cell where the upload data begins 
wks_name = 'webstories'
cell_of_start_df = 'a1'
# upload the dataframe of the clients we want to delete
d2g.upload(webstories_combined,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


The sheet is updated successfully


# SHOWCASE

In [83]:
#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key('1ItBd7UDJPlKq-k71vzTLDxCmjIVcla48NG3Umd89Zow')
sheet = workbook.worksheet('Google UTM')
values = sheet.get_all_values()
showcase = pd.DataFrame(values[1:], columns = values[0])
showcase['Day'] = pd.to_datetime(showcase['Day'])
showcase['Pageviews'] = showcase['Pageviews'].astype(int)
############################################################################
showcase_thisweek = showcase.copy(deep=True)
start_date = ((datetime.datetime.now() - timedelta(days=datetime.datetime.now().weekday() +1)) - timedelta(days=7))
end_date = (datetime.datetime.now() - timedelta(days=datetime.datetime.now().weekday() +1))
mask = (showcase_thisweek['Day'] > start_date) & (showcase_thisweek['Day'] <= end_date) 
showcase_thisweek = showcase_thisweek.loc[mask]
showcase_thisweek = showcase_thisweek.groupby('Product').sum().reset_index()
showcase_thisweek.insert(0, 'Week', 'thisweek') 
############################################################################
showcase_priorweek = showcase.copy(deep=True)
start_date = ((datetime.datetime.now() - timedelta(days=datetime.datetime.now().weekday() +1)) - timedelta(days=14))
end_date =   ((datetime.datetime.now() - timedelta(days=datetime.datetime.now().weekday() +1)) - timedelta(days=7))
mask = (showcase_priorweek['Day'] > start_date) & (showcase_priorweek['Day'] <= end_date) 
showcase_priorweek = showcase_priorweek.loc[mask]
############################################################################
showcase_priorweek = showcase_priorweek.groupby('Product').sum().reset_index()
showcase_priorweek.insert(0, 'Week', 'priorweek') 
############################################################################
showcase_combined = pd.concat([showcase_thisweek,showcase_priorweek], sort=False).reset_index()
showcase_combined['Product'] =    pd.np.where(showcase_combined['Product'].str.contains("WSJ"),"Online Journal", showcase_combined['Product'])
############################################################################
# Set the sheet name you want to upload data to and the start cell where the upload data begins 
wks_name = 'showcase'
cell_of_start_df = 'a1'
# upload the dataframe of the clients we want to delete
d2g.upload(showcase_combined,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


The sheet is updated successfully


# ORDERS BY REFERRER

In [84]:
query = '''


WITH articles AS (SELECT
                  DISTINCT
                  visit_referrer,
                  timestamp(date_time) as read_time,
                  IF(REGEXP_CONTAINS(page_url, 'utm_medium=news_tab') AND NET.REG_DOMAIN(visit_referrer) = 'facebook.com', 1, 0) as news_tab,
                  IF(REGEXP_CONTAINS(post_evar56, 'personalized'), 1, 0) as locked,
                  IF(post_page_event = '0' AND (CAST(exclude_hit AS INT64) <= 0 AND hit_source NOT IN ('5', '8', '9')),1,0) as views,
                  IF(post_prop27 IN ('WSJ_sub_yes'), 1, 0) as sub_status,
                  CONCAT(post_visid_high, post_visid_low) as unique_id,
                  visit_num,
                  IF(prop1 = 'Article', 1, 0) as article,
                  visit_page_num,
                  IF(REGEXP_CONTAINS(visit_start_pagename, 'Article'), 1, 0) as article_visit,
                  IF(page_event_var2 = 'WSJ_Article_social_share', 1, 0) as share,
                  prop10
                  FROM
                   `djomniture.cipomniture_djglobal.*` 
                  WHERE REGEXP_CONTAINS(_TABLE_SUFFIX,r'^\d{4}$*')
    AND _TABLE_SUFFIX BETWEEN '2022_01' AND '2022_12'
    AND DATE(date_time) BETWEEN DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -7 DAY), DAY)) AND DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -1 DAY), DAY))
                  AND SAFE_CAST(exclude_hit AS INT64) <= 0
                  AND hit_source NOT IN ('5',
                                         '8',
                                         '9')
                  AND (REGEXP_CONTAINS(page_url, 'wsj[.com|.net]')
                  OR channel IN ('Online Journal'))),
sales as (SELECT 
          CONCAT(post_visid_high, post_visid_low) as unique_id,
          timestamp(date_time) as subscribe_time,
          post_evar39 as conversion_headline,
          visit_num,
          post_purchaseid,
          FROM
          `djomniture.cipomniture_djglobal.*`
          WHERE REGEXP_CONTAINS(_TABLE_SUFFIX,r'^\d{4}$*')
    AND _TABLE_SUFFIX BETWEEN '2022_01' AND '2022_12'
    AND DATE(date_time) BETWEEN DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -7 DAY), DAY)) AND DATE(DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -1 DAY), DAY))
          AND SAFE_CAST(exclude_hit AS INT64) <= 0
          AND hit_source NOT IN ('5',
                                 '8',
                                 '9')
          AND REGEXP_CONTAINS(post_event_list, r'^1,|,1,|,1$')
          --Relevant order filters go here (i.e. excluding student)
          AND REGEXP_CONTAINS(page_url, 'store.wsj.com|buy.wsj.com')
          AND channel IN ('Online Journal')),
performance as (SELECT
CASE WHEN articles.prop10 IN ('MW_mw_RHF','_mw_RHF','WSJ_wsj_RHF','BOL_bar_RHF','cashapprss','WSJ_Euronews','WSJ_wsj_square','_wsj_square') THEN prop10
WHEN news_tab = 1 THEN "News Tab"
WHEN NET.REG_DOMAIN(articles.visit_referrer) IS NULL THEN 'wsj.com'
ELSE NET.REG_DOMAIN(articles.visit_referrer) END as referrer,
#SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(EXTRACT(date FROM read_time) as STRING),1,8),'01') as DATE) as month,
COUNT(DISTINCT articles.unique_id) as uniques,
COUNT(articles.views) as views,
COUNT(DISTINCT CONCAT(articles.unique_id, articles.visit_num)) as visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) >1, CONCAT(articles.unique_id), NULL)) as twoplus,
COUNT(DISTINCT IF(articles.visit_num = '1', CONCAT(articles.unique_id, articles.visit_num), NULL)) as cold_visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) BETWEEN 2 AND 4, CONCAT(articles.unique_id, articles.visit_num), NULL)) as medium_visits,
COUNT(DISTINCT IF(SAFE_CAST(articles.visit_num AS INT64) >= 5, CONCAT(articles.unique_id, articles.visit_num), NULL)) as hot_visits,
COUNT(DISTINCT IF(articles.share = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as shares,
COUNT(DISTINCT IF(articles.visit_num = sales.visit_num, post_purchaseid, null)) as orders_same_visit,
COUNT(DISTINCT post_purchaseid) as orders_7_days,
COUNT(DISTINCT post_purchaseid) as purchases,
COUNT(DISTINCT articles.unique_id) as unique_ids,
COUNT(DISTINCT post_purchaseid)  / COUNT(DISTINCT articles.unique_id) as conversion_rate,
SAFE_DIVIDE(COUNT(DISTINCT IF(article = 1 AND locked = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)), COUNT(DISTINCT IF(article = 1, CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL))) as percent_locked,
COUNT(DISTINCT IF(article = 1,CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as all_articles,
COUNT(DISTINCT IF(article = 1 AND locked = 1,CONCAT(articles.unique_id, articles.visit_num, articles.visit_page_num), NULL)) as locked_articles
FROM
articles
LEFT JOIN
sales ON 
articles.unique_id = sales.unique_id
AND articles.read_time < sales.subscribe_time
AND DATE_DIFF(DATE(sales.subscribe_time), DATE(articles.read_time), DAY) < 7
WHERE articles.sub_status = 0
AND articles.article_visit = 1
GROUP BY
referrer),
return_visits as (SELECT
CASE WHEN articles.prop10 IN ('MW_mw_RHF','_mw_RHF','WSJ_wsj_RHF','BOL_bar_RHF','cashapprss','cashapprss','WSJ_Euronews','WSJ_wsj_square','_wsj_square') THEN articles.prop10
WHEN articles.news_tab = 1 THEN "News Tab"
WHEN NET.REG_DOMAIN(articles.visit_referrer) IS NULL THEN 'wsj.com'
ELSE NET.REG_DOMAIN(articles.visit_referrer) END as referrer,
#SAFE_CAST(CONCAT(SUBSTR(SAFE_CAST(EXTRACT(date FROM articles.read_time) as STRING),1,8),'01') as DATE) as month,
COUNT(DISTINCT IF(returns.read_time > articles.read_time AND DATE_DIFF(DATE(returns.read_time), DATE(articles.read_time), DAY) < 7
AND SAFE_CAST(returns.visit_num as INT64) > 1, CONCAT(articles.unique_id, articles.visit_num), NULL)) as returns
FROM
articles
LEFT JOIN
articles as returns ON 
articles.unique_id = returns.unique_id
WHERE articles.sub_status = 0
AND articles.article_visit = 1
AND articles.visit_num = '1'
GROUP BY
referrer)
SELECT
*,
SAFE_DIVIDE(twoplus, uniques) as return_rate
FROM
performance
LEFT JOIN
return_visits
USING(referrer)
WHERE uniques >= 1000
ORDER BY
uniques DESC

'''

orders = query_calls(client, query).to_dataframe()


# APPLE NEWS-SPECIFIC TABLE

In [85]:
# PULL IN ALL REPORTS BY REGION FROM THE WSJ FOLDER
allwsjarticledetailimport = pd.concat([pd.read_csv(f) for f in glob.glob('/Users/dicharryd/Documents/ANP Report/News Publisher/monthlies/wsj/*ArticleDetail_all.csv')], ignore_index = True,sort=False)
allwsjarticledetailimport.insert(1,'Region','ALL')
allwsjarticledetailimport.insert(1,'channel','wsj')

allwsjchannelsummaryimport = pd.concat([pd.read_csv(f) for f in glob.glob('/Users/dicharryd/Documents/ANP Report/News Publisher/monthlies/wsj/*ChannelSummary_all.csv')], ignore_index = True,sort=False)
allwsjchannelsummaryimport.insert(1,'Region','ALL')
allwsjchannelsummaryimport.insert(1,'channel','wsj')

allwsjnotificationsimport = pd.concat([pd.read_csv(f) for f in glob.glob('/Users/dicharryd/Documents/ANP Report/News Publisher/monthlies/wsj/*Notifications_all.csv')], ignore_index = True,sort=False)
allwsjnotificationsimport.insert(1,'Region','ALL')
allwsjnotificationsimport.insert(1,'channel','wsj')


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [86]:
allwsjchannelsummaryimport = pd.concat([pd.read_csv(f) for f in glob.glob('/Users/dicharryd/Documents/ANP Report/News Publisher/monthlies/wsj/*ChannelSummary_all.csv')], ignore_index = True,sort=False)
allwsjchannelsummaryimport.insert(1,'Region','ALL')
allwsjchannelsummaryimport.insert(1,'channel','wsj')


#### Daily

In [87]:
allwsjchannelsummary = allwsjchannelsummaryimport
allwsjchannelsummary['Actual Date'] = pd.to_datetime(allwsjchannelsummary['Date'],infer_datetime_format=True)
allwsjchannelsummary['Date'] = pd.to_datetime(allwsjchannelsummary['Date'],infer_datetime_format=True)
allwsjchannelsummary = allwsjchannelsummary.groupby(['Actual Date','Region','channel']).resample('W-Sun', on='Date').sum().reset_index()
allwsjchannelsummary = allwsjchannelsummary.rename(columns={'Date':'Week Ending'})
allwsjchannelsummary['MAU Subs Date'] = (allwsjchannelsummary['Actual Date'] == allwsjchannelsummary['Week Ending'])
allwsjchannelsummary['MAU Subs Date'] = allwsjchannelsummary['MAU Subs Date'].astype(int)
allwsjchannelsummary['Month'] = allwsjchannelsummary['Actual Date'].dt.strftime('%B')
allwsjchannelsummary['Week Dates'] = (allwsjchannelsummary['Week Ending'] - timedelta(days=6)).astype(str) + " - " + allwsjchannelsummary['Week Ending'].astype(str)
allwsjchannelsummary.insert(1,'Timeframe','daily')
allwsjchannelsummary = allwsjchannelsummary[['Month','Week Dates','Actual Date','Week Ending','Timeframe','MAU Subs Date','channel','Region','Total Views','Unique Viewers','Reach','Unique Viewers, Subscribers, All Content','Total Engaged Minutes, Apple News+ Subscribers','MAU, Apple News+ Subscribers']].reset_index(drop=True)


#### Weekly

In [88]:
allwsjchannelsummaryweekly = allwsjchannelsummary.groupby(['Week Dates','Week Ending','channel','Region']).sum().reset_index()
allwsjchannelsummaryweekly.insert(1,'Timeframe','weekly')
allwsjchannelsummaryweekly = allwsjchannelsummaryweekly[['Week Dates','Week Ending','Timeframe','channel','Region','Total Views','Total Engaged Minutes, Apple News+ Subscribers']].sort_values(by='Week Ending',ascending=False).reset_index()


In [89]:
mau = allwsjchannelsummary.copy(deep=True) 
mau = allwsjchannelsummary.loc[allwsjchannelsummary['MAU Subs Date'] == 1]
mau = mau[['Actual Date','MAU, Apple News+ Subscribers']]

In [90]:
allwsjchannelsummary2 = pd.merge(allwsjchannelsummaryweekly, mau, how='left', left_on=['Week Ending'], right_on=['Actual Date'])


In [91]:
#Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key('1Q_mLCBtjCDN0v3xXZPIvHWRxOA8-GZMZSzn_4S4AQFw')
#Selecting which sheet to pulling the data
sheet = workbook.worksheet('Sheet1')
#Pulling the data and transform it to the data frame
values = sheet.get_all_values()
fiscal = pd.DataFrame(values[1:], columns = values[0])
#webstories = pd.DataFrame(values, columns =[0])
fiscal['actual_date'] = pd.to_datetime(fiscal['actual_date'])

In [92]:
combined = pd.merge(allwsjchannelsummary2, fiscal, how='left', left_on=['Week Ending'], right_on=['actual_date'])

In [93]:
# Configure the connection 
scope = ['https://spreadsheets.google.com/feeds']

# Give the path to the Service Account Credential json file 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/dicharryd/service_token.json',
                                                               scope
                                                              )
# Authorise your Notebook
gc = gspread.authorize(credentials)

# The sprad sheet ID, which can be taken from the link to the sheet
spreadsheet_key = '1VpimYGT1nZ20KmsvR6ILs1dIkC4mlDE_0uMsz6ckCDk'

# Set the sheet name you want to upload data to and the start cell where the upload data begins 
wks_name = 'new_dashboard'
cell_of_start_df = 'a1'
# upload the dataframe of the clients we want to delete
d2g.upload(combined,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

The sheet is updated successfully


# TOP STORIES

In [94]:
pd.set_option('display.max_columns', 100)
features = allwsjarticledetailimport.copy(deep=True)
features['Date']= pd.to_datetime(features['Date'])
features['date']= pd.to_datetime(features['Date'])
features = features[['Date','Where Featured by Apple','date']]
features = features[features['Where Featured by Apple'].notna()] #drop only NaN rows

start_date = (pd.to_datetime("now") - pd.offsets.MonthBegin(14))
end_date = pd.to_datetime("now")
mask = (features['Date'] > start_date) & (features['Date'] <= end_date) 
features = features.loc[mask]

#features.sort_values(by='Date',ascending=False)

In [95]:
#CLASSIFY FEATURE TYPE

topstories = features[features['Where Featured by Apple'].str.contains('Featured in Top Stories', na=False)]
topstories.insert(1,'Feature','Top Stories')
topstories.insert(1,'Feature Order','1')
#topstories = topstories[['Region','Date','Feature Order','Feature','Article']]

stocks = features[features['Where Featured by Apple'].str.contains('Featured in Stocks Top Stories', na=False)]
stocks.insert(1,'Feature','Stocks Top Stories')
stocks.insert(1,'Feature Order','2')
#stocks = stocks[['Region','Date','Feature Order','Feature','Article']]

trending = features[features['Where Featured by Apple'].str.contains('Appeared in Trending Stories', na=False)]
trending.insert(1,'Feature','Trending Stories')
trending.insert(1,'Feature Order','3')
#trending = trending[['Region','Date','Feature Order','Feature','Article']]

email = features[features['Where Featured by Apple'].str.contains('Featured in Apple Email', na=False)]
email.insert(1,'Feature','Apple Email')
email.insert(1,'Feature Order','4')
#email = email[['Region','Date','Feature Order','Feature','Article']]

spotlight = features[features['Where Featured by Apple'].str.contains('Featured in Spotlight', na=False)]
spotlight.insert(1,'Feature','Spotlight')
spotlight.insert(1,'Feature Order','5')
#spotlight = spotlight[['Region','Date','Feature Order','Feature','Article']]

boosted = features[features['Where Featured by Apple'].str.contains('Boosted by Apple Editors', na=False)]
boosted.insert(1,'Feature','Boosted by Apple Editors')
boosted.insert(1,'Feature Order','6')
#boosted = boosted[['Region','Date','Feature Order','Feature','Article']]

takeover = features[features['Where Featured by Apple'].str.contains('Featured as Today View Takeover', na=False)]
takeover.insert(1,'Feature','Today View Takeover')
takeover.insert(1,'Feature Order','7')
#takeover = takeover[['Region','Date','Feature Order','Feature','Article']]

newsplusfeed = features[features['Where Featured by Apple'].str.contains('Article Featured in News', na=False)]
newsplusfeed.insert(1,'Feature','Article Featured in News+ Feed')
newsplusfeed.insert(1,'Feature Order','8')
#newsplusfeed = newsplusfeed[['Region','Date','Feature Order','Feature','Article']]

video = features[features['Where Featured by Apple'].str.contains('Video Featured in Today Feed', na=False)]
video.insert(1,'Feature','Video Featured in Today Feed')
video.insert(1,'Feature Order','9')
#video = video[['Region','Date','Feature Order','Feature','Article']]

articlefeat = features[features['Where Featured by Apple'].str.contains('Article Featured in Today Feed', na=False)]
articlefeat.insert(1,'Feature','Article Featured in Today Feed')
articlefeat.insert(1,'Feature Order','10')
#articlefeat = articlefeat[['Region','Date','Feature Order','Feature','Article']]

editorial = features[features['Where Featured by Apple'].str.contains('Editorial', na=False)]
editorial.insert(1,'Feature','Featured in an Editorial Group')
editorial.insert(1,'Feature Order','11')
#editorial = editorial[['Region','Date','Feature Order','Feature','Article']]

local = features[features['Where Featured by Apple'].str.contains('Local', na=False)]
local.insert(1,'Feature','Featured in Local News')
local.insert(1,'Feature Order','12')
#local = local[['Region','Date','Feature Order','Feature','Article']]

audiostory = features[features['Where Featured by Apple'].str.contains('Audio Story Published', na=False)]
audiostory.insert(1,'Feature','Audio Story Published')
audiostory.insert(1,'Feature Order','13')
#audiostory = audiostory[['Region','Date','Feature Order','Feature','Article']]

audiofeed = features[features['Where Featured by Apple'].str.contains('Recommended in Audio Feed', na=False)]
audiofeed.insert(1,'Feature','Recommended in Audio Feed')
audiofeed.insert(1,'Feature Order','14')
#audiofeed = audiofeed[['Region','Date','Feature Order','Feature','Article']]

newsbriefing = features[features['Where Featured by Apple'].str.contains('Featured in ‚ÄúApple News Today‚Äù Briefing', na=False)]
newsbriefing.insert(1,'Feature','Featured in News Briefing')
newsbriefing.insert(1,'Feature Order','15')
#newsbriefing = newsbriefing[['Region','Date','Feature Order','Feature','Article']]

specialtopic = features[features['Where Featured by Apple'].str.contains('Featured in Special Topic', na=False)]
specialtopic.insert(1,'Feature','Featured in Special Topic')
specialtopic.insert(1,'Feature Order','16')
#specialtopic = specialtopic[['Region','Date','Feature Order','Feature','Article']]


In [96]:
topstories = pd.concat([topstories, stocks, trending, email, spotlight, boosted, takeover, newsplusfeed, video, articlefeat, editorial, local, audiostory, audiofeed, newsbriefing, specialtopic], sort=False)
topstories = pd.merge(topstories, fiscal, how='left', left_on=['Date'], right_on=['actual_date'])
topstories = topstories.loc[topstories['Feature'] == 'Top Stories']
topstories = topstories.groupby(['DJFWID','DJFW','Feature']).count().reset_index()
topstories.rename(columns={'Date': 'Count'}, inplace=True) #RENAME COLUMN
topstories = topstories[['DJFWID','DJFW','Feature','Count']].reset_index().sort_values(by='DJFWID',ascending=False)

# The sprad sheet ID, which can be taken from the link to the sheet
spreadsheet_key = '1VpimYGT1nZ20KmsvR6ILs1dIkC4mlDE_0uMsz6ckCDk'

# Set the sheet name you want to upload data to and the start cell where the upload data begins 
wks_name = 'top_stories'
cell_of_start_df = 'a1'
# upload the dataframe of the clients we want to delete
d2g.upload(topstories,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')


The sheet is updated successfully


# NOTIFICATIONS

In [97]:
allwsjnotifications = allwsjnotificationsimport.copy(deep=True)
allwsjnotifications['Date']= pd.to_datetime(allwsjnotifications['Date'])
allwsjnotifications = pd.merge(allwsjnotifications, fiscal, how='left', left_on=['Date'], right_on=['actual_date'])
allwsjnotifications = allwsjnotifications[['DJFWID','Article']]
allwsjnotifications.rename(columns={'Article': 'Count'}, inplace=True) #RENAME COLUMN
allwsjnotifications = allwsjnotifications.groupby('DJFWID').count().reset_index().sort_values(by='DJFWID',ascending=False)

# The sprad sheet ID, which can be taken from the link to the sheet
spreadsheet_key = '1VpimYGT1nZ20KmsvR6ILs1dIkC4mlDE_0uMsz6ckCDk'

# Set the sheet name you want to upload data to and the start cell where the upload data begins 
wks_name = 'notifications'
cell_of_start_df = 'a1'
# upload the dataframe of the clients we want to delete
d2g.upload(allwsjnotifications,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

The sheet is updated successfully
